In [1]:
import cv2
import mediapipe as mp
import urllib.request
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
import PyQt5
from PIL import Image
from IPython.display import Video
import nb_helpers
import time
import random as rand
import os

In [27]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
#mp_face_mesh = mp.solutions.face_mesh

In [28]:
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

In [29]:
from win32api import GetSystemMetrics
w =  GetSystemMetrics(0)
h = GetSystemMetrics(1)
print("Width =", GetSystemMetrics(0))
print("Height =", GetSystemMetrics(1))

Width = 1280
Height = 720


In [30]:
def inImage(image,points):
    for point in points:
        x,y = point
        im = Image.open(image) # Can be many different formats.
        pix = im.load()
        colour = pix[x,y]
        #print(colour)
        if colour != (0,0,0):
            return False
    
    return True

#print(inImage("pgreen.jpg",[(150,150),(0,0)]))

In [31]:
path = "C:\Computer\Programming\opencv\c\poses"
pos = os.listdir(path)
poses = []
for p in pos:
    if p.endswith("jpg"):
        poses.append(p)
print(poses)

['pose2.jpg', 'pose3.jpg', 'pose4.jpg', 'pose5.jpg']


In [32]:
def drawline(layer,a,b,colour,width):
    cv2.line(layer, (int(a.x*w), int(a.y*h)), (int(b.x*w), int(b.y*h)), colour, width)

In [33]:
def centercoor(text,scale,thicknes):
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = scale
    thickness = thicknes
    text_width, text_height = cv2.getTextSize(text, fontFace, fontScale, thickness)[0]
    return (int(w / 2) - int(text_width / 2), int(h / 2) + int(text_height / 2))

In [38]:
cap = cv2.VideoCapture(0)

currimage = os.path.join(path,rand.choice(poses))
start = True

while True:
    #print("Running")

    landDetected = True

    inArea = True

    curTime = time.time()

    success, img = cap.read()

    img = cv2.resize(img, (w,h))
    img = cv2.flip(img, 1)

    #imgPlayer = np.zeros((512,512,3), np.uint8)
    #imgPlayer = cv2.resize(imgPlayer, (w,h))
    imgPlayer = img

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(imgRGB)
    image = cv2.imread(currimage)

    try: 
        lm = results.pose_landmarks.landmark[11:17] + results.pose_landmarks.landmark[19:21]
    except: 
        landDetected = False

    if landDetected: 
        error = int(w / 13)
        #print("landDetected")
        


        lm = results.pose_landmarks.landmark

        lShould = lm[12]
        lElb = lm[14]
        lWrist = lm[16]
        lInd = lm[20]
        lWaist = lm[24]

        rShould = lm[11]
        rElb = lm[13]
        rWrist = lm[15]
        rInd = lm[19]
        rWaist = lm[23]

        head = lm[0]

        

        limbs = [lShould,lElb,lWrist,lInd,rShould,rElb,rWrist,rInd,rWaist,lWaist,head]
        points = [(l.x,l.y) for l in limbs]
        
        # Create the condition to check if each pixel in the original image is black
        overlay = cv2.resize(cv2.imread(currimage),(w,h))
        overlay = cv2.flip(overlay,1)

        imgPlayer = cv2.addWeighted(imgPlayer,0.4,overlay,0.1,0)

        blue = (255,0,0)
        linewidth = 5
        #cv2.line(imgPlayer, (int(lShould.x*w), int(lShould.y*h)), (int(lElb.x*w), int(lElb.y*h)), (255,0,0), 5)
        drawline(imgPlayer,lShould,lElb,blue,linewidth)
        #cv2.line(imgPlayer, (int(lElb.x*w), int(lElb.y*h)), (int(lWrist.x*w), int(lWrist.y*h)), (255,0,0), 5)
        drawline(imgPlayer,lWrist,lElb,blue,linewidth)
        #cv2.line(imgPlayer, (int(lWrist.x*w), int(lWrist.y*h)), (int(lInd.x*w), int(lInd.y*h)), (255,0,0), 5)
        drawline(imgPlayer,lWrist,lInd,blue,linewidth)

        #cv2.line(imgPlayer, (int(rShould.x*w), int(rShould.y*h)), (int(rElb.x*w), int(rElb.y*h)), (255,0,0), 5)
        drawline(imgPlayer,rShould,rElb,blue,linewidth)
        #cv2.line(imgPlayer, (int(rElb.x*w), int(rElb.y*h)), (int(rWrist.x*w), int(rWrist.y*h)), (255,0,0), 5)
        drawline(imgPlayer,rWrist,rElb,blue,linewidth)
        #cv2.line(imgPlayer, (int(rWrist.x*w), int(rWrist.y*h)), (int(rInd.x*w), int(rInd.y*h)), (255,0,0), 5)
        drawline(imgPlayer,rWrist,rInd,blue,linewidth)

        #cv2.line(imgPlayer, (int(lWaist.x*w), int(lWaist.y*h)), (int(rWaist.x*w), int(rWaist.y*h)), (255,0,0), 5)
        drawline(imgPlayer,rWaist,lWaist,blue,linewidth)
        drawline(imgPlayer,rShould,lShould,blue,linewidth)
        drawline(imgPlayer,rShould,rWaist,blue,linewidth)
        drawline(imgPlayer,lShould,lWaist,blue,linewidth)

        #drawline(imgPlayer,head,lShould,blue,linewidth)
        #drawline(imgPlayer,head,rShould,blue,linewidth)

        if start:
            currimage = os.path.join(path,"default\\start.jpg")
            cv2.putText(imgPlayer, "Line up to start", (centercoor("Line up to start",3,3)[0],75), cv2.FONT_HERSHEY_SIMPLEX,3,(255,255,255), 3)

        isin = True
        for point in points:
            x,y = point
            if x <= 0 or x >= 1 or y <= 0 or y >=1:
                continue
            x = int(x*w)
            y = int(y*h)
            im = cv2.imread(currimage) # Can be many different formats.
            im = cv2.flip(im, 1)

            if (im[y,x] == [0]).all():
                cv2.circle(imgPlayer, (x, y), 5, (0,255,0), cv2.FILLED)
            else:
                cv2.circle(imgPlayer, (x, y), 5, (0,0,255), cv2.FILLED)
                isin = False

        if isin:
            if start:
                start = False
            currimage = os.path.join(path,rand.choice(poses))
            

        #shows user their fps
        

    cv2.imshow("Image Player", imgPlayer)

    if cv2.waitKey(1)==ord('q'):
        break

cv2.destroyAllWindows()
cap = None